In [2]:
import sys
import os
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt  # graphic library, for plots
import numba as nb
import filedialogs

In [21]:
import numpy as np
import bokeh.sampledata as sampledata
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook

In [15]:
output_notebook() 

Loading BokehJS ...

In [4]:
import pandas

### Importing Data

In [5]:
left_trace = pandas.read_csv('example_data/fish_rbc_uv_test2_63x_500Hz_Left.csv')
right_trace = pandas.read_csv('example_data/fish_rbc_uv_test2_63x_500Hz_Right.csv')

In [7]:
right_trace.head()

,Slice,Mean
0,1,0.276
1,2,0.299
2,3,0.506
3,4,0.345
4,5,0.391


In [11]:
left_trace = left_trace.rename(columns={"Slice": "Frame", "Mean": "Left"})
right_trace = right_trace.rename(columns={"Slice": "Frame", "Mean": "Right"})

In [13]:
right_trace['Right'].head()

0    0.276
1    0.299
2    0.506
3    0.345
4    0.391
Name: Right, dtype: float64

In [30]:
df = left_trace.merge(right_trace, left_on='Frame', right_on='Frame')

In [32]:
df.head()

,Frame,Left,Right
0,1,0.2760,0.276
1,2,0.7589,0.299
2,3,0.4830,0.506
3,4,0.3910,0.345
4,5,0.4370,0.391


In [33]:
df.Frame

0          1
1          2
2          3
3          4
4          5
        ... 
4002    4003
4003    4004
4004    4005
4005    4006
4006    4007
Name: Frame, Length: 4007, dtype: int64

In [29]:
df = pandas.concat([left_trace, right_trace['Right'], axis=1)

SyntaxError: invalid syntax (1629631498.py, line 1)

In [27]:
type(right_trace['Right'])

pandas.core.series.Series

In [41]:
p = figure(width=950, height=400, title="", tools="", match_aspect=False)
p.line(df.Frame, df.Left, color="red", alpha=0.4, line_width=1)
p.line(df.Frame, df.Right, color="blue", alpha=0.4, line_width=1)
show(p)

In [42]:
df.Left.mean()

1.457608210631395

In [43]:
df.Right.mean()

1.3004354629398553

In [ ]:
df_norm = 

In [45]:
df.Left = df.Left/df.Left.mean()

In [46]:
df.Right = df.Right/df.Right.mean()

In [47]:
p = figure(width=950, height=400, title="", tools="", match_aspect=False)
p.line(df.Frame, df.Left, color="red", alpha=0.4, line_width=1)
p.line(df.Frame, df.Right, color="blue", alpha=0.4, line_width=1)
show(p)

In [71]:
p = figure(width=950, height=400, title="", tools="", match_aspect=False)
p.line(df.Frame, df.Left, color="red", alpha=0.4, line_width=1)
p.line(df.Frame, df.Right, color="blue", alpha=0.4, line_width=1)
p.line(df.Frame, 10*left_above_thresh, color="green", alpha=0.8, line_width=1)
show(p)

### Approach

1) Detect when signal is above threshold in one of the channels
2) Create masking timeseries -> with a passing RBC
3) For every rbc, compare the ratio of left vs. right total counts

In [60]:
left_above_thresh = df.Left > 1

In [61]:
left_above_thresh

0       False
1       False
2       False
3       False
4       False
        ...  
4002    False
4003    False
4004    False
4005    False
4006    False
Name: Left, Length: 4007, dtype: bool

In [70]:
p = figure(width=950, height=400, title="", tools="", match_aspect=False)
p.line(df.Frame, df.Left, color="red", alpha=0.4, line_width=1)
p.line(df.Frame, df.Right, color="blue", alpha=0.4, line_width=1)
p.line(df.Frame, 10*left_above_thresh, color="green", alpha=0.8, line_width=1)
show(p)

In [66]:
def schmitt_trigger(lo, hi, cb, value):
    """http://en.wikipedia.org/wiki/Schmitt_trigger"""
    while True:
        newvalue = (yield value)
        if newvalue is None:
            raise StopIteration()
        if newvalue >= hi and value < hi:
            cb(hi, newvalue, 'to-hi')
        elif newvalue <= lo and value > lo:
            cb(lo, newvalue, 'to-lo')
        value = newvalue

def test():
    i = [1, 2, 3, 4, 5, 4, 2, 6, 3]
    r = []
    def cb(*a):
        signal, value = a[:2]
        r.append(value)
        pass

    lo = 2
    hi = 5
    g = schmitt_trigger(lo, hi, cb, 0)
    g.next()
    for x in i:
        g.send(x)
    assert r == [5, 2, 6], str(r)

AttributeError: 'generator' object has no attribute 'next'

In [49]:
p = figure(width=950, height=400, title="", tools="", match_aspect=False)
p.line(df.Frame, df.Left/df.Right, color="red", alpha=0.4, line_width=1)
show(p)

### Learning Bokeh again

In [24]:
x = np.linspace(-6, 6, 500)
y = 8*np.sin(x)*np.sinc(x)

p = figure(width=800, height=300, title="", tools="", match_aspect=True)

p.line(df., y, color="navy", alpha=0.4, line_width=4)
p.background_fill_color = "#efefef"
p.xaxis.fixed_location = 0
p.yaxis.fixed_location = 0

show(p)

In [26]:
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

p = figure(title="simple line example", height=300, width=600, y_range=(-5,5),
           toolbar=None,background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()
    
show(p, notebook_handle=True)

interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))

ValueError: failed to validate figure(id='p1271', ...).toolbar: expected an instance of type Toolbar, got None of type NoneType